### **5.15 - [Practica] Installing Airflow with Rancher and the Kubernetes Executor**

In the previous video we have set up our Kubernetes Cluster,  now it’s time to install Airflow on it. To do this  we are going use Rancher as well as Helm charts to install everything without too much pain. A Helm chart  is a collection of files that describe a related set of Kubernetes resources.  A single chart might be used to deploy simple, like a memcached pod, or something  complex like Airflow with web server, a database and so on. Helm charts are installed using Helm which  is binary helping to install and manage Kubernetes applications using Helm charts. In the case of Rancher,  there is like an overlay named “Catalog”. Basically, a catalog is just a collection of Helm charts.  Alright, let’s move to the practice so that it will be clearer for you. From the header, if you hover  “airflow”,  you can see a sort of hierarchy, with Global, and then your cluster “airflow” with projects named “Default”  and “System”.  While rancher helps you to manage your Kubernetes cluster,  It can actually deal with many clusters.  For example, you can install apps on a specific cluster or globally.  Let’s click on “Global”.  Here, we have our cluster named “airflow”, but we are going to install a new App on this cluster. To do this,  click on “Apps” here. Then, “Manage Catalogs”.  Here, you have different repositories with helm charts.  In our case, we are going to add a new catalog having only Airflow with settings preset. Click on “Add  Catalog”. Type “airflow-charts” for the name.  Then the catalog link is “https://  github.com  /marclamberti  /airflow-  helm-chart.git”.  Alright, now click on “Create”. The catalog “airflow-charts” has been activated. Click on “Apps” again,  then “Launch”.  And here, you have a list of applications that you can set up using Rancher and Helm divided by catalog.  So you have grafana, mongodb, mysql and so on.  Do not hesitate to take a look at this list,  There is many apps available that you can install easily.  All right,  click on the chart named “airflow-k8s”.  From here you can access to the detailed description of the chart describing how to use it,  what are the parameters available  and so on.  Next you have “configuration options” that we gonna set in a minute.  The section “members” is used to narrow the access of the application. And finally, “answer overrides” that  can be used to add parameters to the chart.  So, from the configuration options we set the name to “airflow”.  The template version must be set to 4.4.0.  The target project is “default”.  We allow the app to access and manage all resources in the cluster by checking “Cluster”.  Now the chart is configured  we can start it by clicking on “Launch”.  So Airflow is getting installed on the cluster as you can see here.  Click on it. If you scroll down, here, you can see the pods corresponding to the components of Airflow such  as the web server and the scheduler being started by Kubernetes.  This process can take a lot of time like around 7 minutes before all the pods turn active and are running.  If you don’t get any errors below the name of a given pod, then it is still trying to be run,  no worries. I’m gonna pause the video right now and I will come back as soon as Airflow is ready. Ok  at this point the metadatabase, the scheduler and the web server should be active and running as shown  from the scale column.  Let's click on the web server.  . Here, you get an overview of the deployment object used for running the web server.  Basically you get information such as the running pods, only one in our case, the events, the environment  variables set, the volumes used and so on.  From the pods you can see that one pod is active with the following name.  The docker image used is latest version of marclambertiml/airflow. Notice the “+ 1 image” next to the  image name.  You will understand what it means in a minute.  The pod is running on the following worker with this ip address and has never been restarted.  That’s great but we can get even more information on the pod by clicking on it. At the top of the  page  you have its name, in which namespace it is running, its IP address, when it was created. Then you have  different panels,  the containers, events, status and so on.  Let’s focus on the containers panel. Here  you can see that we have two containers. One terminated and one running with both different docker images.  Just below the name of the terminated container,  you can read “Init container”.  So what does that mean?  Well, when you run Airflow with the Kubernetes Executor there are several options for synchronizing  your DAGs.  The easiest way is to use init-container.  Init-containers are containers that run before the app containers are started.  They always run to completion and each init container must complete successfully before the next one  starts.  If a Pod’s init container fails, Kubernetes repeatedly restarts the Pod until the init container succeeds.  Init-containers are used to initialize the app containers. In the context of Airflow,  each time a pod is started, an init-container run in first and pull the DAGs from a GitHub repository  so that the following container running a component of Airflow can access the latest DAGs.  If you click on the icon on the right, then “view logs”,  you can see that a repository have been cloned into the dags folder.  Now the question is, which repository?  Well to answer this question, close the logs, go to “Airflow”, “Global”, “Apps”.  By the way,  this is where you can delete, clone, upgrade the app if you want.  Don’t forget that the app was installed globally so you have to be in “global scope” if you want  access to these commands.  Click on “airflow” just below “target projects”. And if you scroll down until your reach “Config Maps”,  click on the first one and you get all the environment variables set to configure Airflow. To specify  the git repository that we want to pull the DAGs from,  we need to set three variables. “AIRFLOW__KUBERNETES__GIT_  REPO” with the link of the repo. “AIRFLOW__KUBERNETES__GIT_  BRANCH” with the branch we want to clone which is master in our case. And Finally, AIRFLOW__  KUBERNETES__GIT_DAGS_FOLDER_MOUNT_  POINT corresponding to where the DAGs will be mounted in the container. We specified the same  path as the one defined by the parameter dags_folder in airflow.cfg. Actually, we  can open a session to the container running the web server to check if the DAGs exist. Click on “Apps”, “airflow”,  then click on the workload corresponding to the web server, and from the little icon the right, click  on it, and “Execute Shell”.  Now you are connected inside the container running the web server. If you type “ls” you should be familiar  with the following files and folders. Display the content of the dags folder with “ls dags”,  and as you can see, our DAGs exist in the container so the init-container method has worked as expected.  Ok terminate the session by clicking on “Close”.  Finally there are two disadvantages with the init-container method. First, worker pods will take a longer time  to start because each time a pod starts, the repository needs to be pulled.  The second drawback is if you have pushed a new definition of a dog still being executed by a pod.  When a new pod is started, it will pull and run a different version of the DAG than the one running in  the older pod and so, you may end up with unexpected behaviors.  Another way of synchronizing your DAG is my mounting a shared persistent volume.  You can store your DAGs on an external volume such as NFS, EFS or AFS for azure file storage.  You will need to set the accessMode of your persistent volume to ReadOnlyMany or ReadWriteMany  in order to share it with multiple pods.  The last way of synchronizing your DAGs is by creating a new Docker Airflow image each time you want  to deploy new DAGs.  From now on, let’s take a quick break and see you in the video where we are going to run our data pipeline  with the Kubernetes Executor.  